# Data and File Formats

ANDES supports multiple file formats for loading power system data, including its native Excel format, PSS/E files, and MATPOWER cases. This tutorial covers loading data from different sources, inspecting and modifying parameters, and saving modified cases.

Understanding how to work with data is essential for practical power system studies, where you often need to adapt standard test cases to specific scenarios or import data from other simulation tools.

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

In [1]:
%matplotlib inline

import andes
import pandas as pd

andes.config_logger(stream_level=20)

# Configure pandas to show all columns
pd.options.display.max_columns = 20

## Loading ANDES Excel Files

The native ANDES Excel format (`.xlsx`) is the best-supported format and is recommended for most use cases. Each sheet in the workbook corresponds to a model type (Bus, Line, GENROU, etc.), and each row represents a device instance.

Use `andes.load()` to create a System object from a case file. The `andes.get_case()` function returns the path to built-in test cases.

In [2]:
ss = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.1131 seconds.


System internal structure set up in 0.0129 seconds.


The System object provides a summary of all loaded models. Each model can be accessed as an attribute of the system, for example `ss.Bus`, `ss.Line`, or `ss.GENROU`.

## Loading PSS/E Files

PSS/E is the industry-standard power system simulation software from Siemens. ANDES can import PSS/E data files directly, supporting RAW files for power flow data and DYR files for dynamic model parameters.

| File Type | Extension | Contents |
|-----------|-----------|----------|
| RAW | `.raw` | Power flow data (buses, generators, loads, branches) |
| DYR | `.dyr` | Dynamic model parameters (exciters, governors, stabilizers) |

To load PSS/E files, pass the RAW file path to `andes.load()` and use the `addfile` argument for the DYR file.

In [3]:
raw_path = andes.get_case('kundur/kundur.raw')
dyr_path = andes.get_case('kundur/kundur_full.dyr')

ss_psse = andes.load(raw_path, addfile=dyr_path)
ss_psse.PFlow.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur.raw"...


  MODIFIED KUNDUR'S TWO-AREA TEST SYSTEM, DISTRIBUTED WITH ANDES


...

0: |F(x)| = 3.175850023


1: |F(x)| = 3.176155228e-08


Converged in 2 iterations in 0.0011 seconds.


Report saved to "kundur_out.txt" in 0.0005 seconds.


True

### Command Line Usage

PSS/E files can also be loaded from the command line:

```bash
# Power flow only (RAW file)
andes run kundur.raw

# With dynamic models (RAW + DYR)
andes run kundur.raw --addfile kundur.dyr -r tds
```

### Converting to ANDES Format

PSS/E files can be converted to the native ANDES Excel format for easier editing:

```bash
andes run kundur.raw --addfile kundur.dyr --convert xlsx
```

## Inspecting Parameters

After loading a system, you can inspect model parameters in several ways. Each model provides a `cache.df` attribute that returns a pandas DataFrame containing all parameter values.

### Viewing Model Parameters

In [4]:
# View GENROU parameters as DataFrame
ss.GENROU.cache.df

,idx,u,name,bus,gen,coi,coi2,Sn,Vn,fn,...,gammaq,xd,xq,xd2,xq1,xq2,Td10,Td20,Tq10,Tq20
uid,,,,,,,,,,,,,,,,,,,,,
0,1,1.0,GENROU_1,1,1,None,None,900.0,20.0,60.0,...,1.0,0.2,0.188889,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05
1,2,1.0,GENROU_2,2,2,None,None,900.0,20.0,60.0,...,1.0,0.2,0.188889,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05
2,3,1.0,GENROU_3,3,3,None,None,900.0,20.0,60.0,...,1.0,0.2,0.188889,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05
3,4,1.0,GENROU_4,4,4,None,None,900.0,20.0,60.0,...,1.0,0.2,0.188889,0.027778,0.061111,0.027778,8.0,0.03,0.4,0.05


The DataFrame shows all parameters for each device, with the unique device index (`idx`) in the first column. Parameters are shown after conversion to per-unit on the system base.

To see the original input values before per-unit conversion, use `cache.df_in`.

In [5]:
# Original input values
ss.GENROU.cache.df_in[['idx', 'Sn', 'M', 'xd', 'xq']]

,idx,Sn,M,xd,xq
uid,,,,,
0,1,900.0,13.00,1.8,1.7
1,2,900.0,13.00,1.8,1.7
2,3,900.0,12.35,1.8,1.7
3,4,900.0,12.35,1.8,1.7


### Getting Specific Parameters

To retrieve a specific parameter value for a device, use the `get()` method. This is useful when you need to read a value for use in calculations or comparisons.

In [6]:
# Get the inertia constant M for generator 1
M_value = ss.GENROU.get('M', 1)
print(f"Inertia M for GENROU 1: {M_value} pu")

Inertia M for GENROU 1: 117.0 pu


### Accessing Parameter Arrays

Each parameter is stored as a `NumParam` object with a `v` attribute containing the numerical values as a NumPy array. This provides direct access for bulk operations.

In [7]:
# Get all M values as an array
print("All generator inertia constants:")
print(ss.GENROU.M.v)

All generator inertia constants:
[117.   117.   111.15 111.15]


## Modifying Parameters

Parameters can be modified using the `alter()` method, which takes the parameter name, device index, and new value. ANDES handles per-unit conversion automatically based on the model's base values.

### Using the alter() Method

In [8]:
# Load a fresh system
ss_mod = andes.load(andes.get_case('kundur/kundur_full.xlsx'))

# Check original value
print(f"Original M: {ss_mod.GENROU.get('M', 1, attr='vin')}")

# Modify the inertia constant
ss_mod.GENROU.alter('M', 1, 10.0)

# Verify the change
print(f"Modified M: {ss_mod.GENROU.get('M', 1, attr='vin')}")

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0179 seconds.


System internal structure set up in 0.0121 seconds.


Original M: 13.0
Modified M: 10.0


The value passed to `alter()` should be in the same units as the input file (before per-unit conversion). ANDES will automatically convert it to the appropriate system base.

### Direct Array Modification

For bulk modifications or performance-critical operations, you can directly modify the parameter arrays. When doing so, always use in-place assignment with slicing to avoid memory issues.

In [9]:
# Scale all PQ loads by 10%
ss_mod.PQ.Ppf.v[:] *= 1.1
ss_mod.PQ.Qpf.v[:] *= 1.1

print("Scaled PQ loads:")
print(ss_mod.PQ.Ppf.v)

Scaled PQ loads:
[0. 0.]


### Refreshing Cached Views

After modifying parameters, the cached DataFrames (`cache.df`, `cache.df_in`) are not automatically updated. Call `cache.refresh()` to update them for inspection.

In [10]:
ss_mod.GENROU.cache.refresh()
ss_mod.GENROU.cache.df_in[['idx', 'M']]

,idx,M
uid,,
0,1,10.00
1,2,13.00
2,3,12.35
3,4,12.35


## Modifying Device Status

Each device has a connectivity status parameter `u` that can be set to 0 (disabled) or 1 (enabled). This is useful for studying contingencies without removing devices from the data file.

To disable a device before running simulation, load with `setup=False`, modify the status, then call `setup()`.

In [11]:
ss_cont = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable a transmission line
ss_cont.Line.alter('u', 'Line_3', 0)

# Complete system setup and run power flow
ss_cont.setup()
ss_cont.PFlow.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0179 seconds.


...

4: |F(x)| = 2.880943096e-05


5: |F(x)| = 3.937117299e-11


Converged in 6 iterations in 0.0029 seconds.


Report saved to "kundur_full_out.txt" in 0.0006 seconds.


True

## Saving Modified Cases

After modifying parameters, you can save the system to a new file. This is useful for creating variations of a base case for parameter studies.

In [12]:
# Save to new Excel file
andes.io.xlsx.write(ss_mod, 'modified_case.xlsx', overwrite=True)

xlsx file written to "modified_case.xlsx"


True

## Accessing Variables After Simulation

After running power flow or time-domain simulation, solved variable values can be accessed through the model objects. Each variable has a `v` attribute containing its current value.

### Power Flow Results

In [13]:
ss.PFlow.run()

# Bus voltage magnitudes
print("Bus voltages (pu):")
print(ss.Bus.v.v)

-> System connectivity check results:


  No islanded bus detected.


  System is interconnected.


  Each island has a slack bus correctly defined and enabled.



-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method


...

3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103979e-07


Converged in 5 iterations in 0.0025 seconds.


Report saved to "kundur_full_out.txt" in 0.0005 seconds.


Bus voltages (pu):
[1.         1.         1.         1.         0.98337472 0.96908585
 0.9562181  0.95400018 0.96856366 0.98377143]


### Variable Types

Variables are classified as either state (differential) or algebraic:

- **State variables** (stored in `ss.dae.x`): Generator angles, speeds, flux linkages
- **Algebraic variables** (stored in `ss.dae.y`): Bus voltages, angles, power injections

Each variable object has an `a` attribute containing its address(es) in the system arrays.

In [14]:
# Generator speed is a state variable
print(f"GENROU.omega type: {type(ss.GENROU.omega).__name__}")
print(f"GENROU.omega addresses: {ss.GENROU.omega.a}")

# Bus voltage is an algebraic variable
print(f"\nBus.v type: {type(ss.Bus.v).__name__}")
print(f"Bus.v addresses: {ss.Bus.v.a}")

GENROU.omega type: State
GENROU.omega addresses: []

Bus.v type: Algeb
Bus.v addresses: [10 11 12 13 14 15 16 17 18 19]


## Working with Device Indices

Each device is identified by a unique index (`idx`). The index can be any hashable type (string, integer, etc.) and is used consistently across ANDES for referencing devices.

### Listing Device Indices

In [15]:
# Get all Line indices
print("Line indices:")
print(ss.Line.idx.v)

Line indices:
['Line_0', 'Line_1', 'Line_2', 'Line_3', 'Line_4', 'Line_5', 'Line_6', 'Line_7', 'Line_8', 'Line_9', 'Line_10', 'Line_11', 'Line_12', 'Line_13', 'Line_14']


In [16]:
# Get all GENROU indices
print("GENROU indices:")
print(ss.GENROU.idx.v)

GENROU indices:
[1, 2, 3, 4]


### Converting Index to Internal UID

Internally, ANDES uses zero-based integer UIDs for array indexing. The `idx2uid()` method converts an external index to its internal UID.

In [17]:
# Convert idx to uid
uid = ss.GENROU.idx2uid(1)
print(f"GENROU idx=1 has uid={uid}")

# Access parameter by uid
print(f"M value at uid {uid}: {ss.GENROU.M.v[uid]}")

GENROU idx=1 has uid=0
M value at uid 0: 117.0


## Parameter Units

Parameters are typically specified in per-unit on the device or system base. Common conventions are:

| Parameter Type | Typical Unit |
|---------------|---------------|
| Power (`p0`, `q0`) | pu on system MVA base |
| Voltage (`v0`) | pu on bus nominal voltage |
| Impedance (`R`, `X`) | pu on system base |
| Inertia (`H`, `M`) | seconds (M = 2H) |
| Time constants | seconds |

Consult the model documentation for specific parameter definitions and units.

## Cleanup

In [18]:
!andes misc -C
!rm -f modified_case.xlsx

"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_out.txt" removed.


## Next Steps

- {doc}`06-plotting-results` - Visualizing simulation results
- {doc}`07-eigenvalue-analysis` - Small-signal stability analysis
- {doc}`08-parameter-sweeps` - Running batch simulations